In [80]:
import csv
import agate
from pprint import pprint
from datetime import datetime
import json
import random
from titlecase import titlecase

In [81]:
local_homes = {}

In [82]:
with open('local_homes_keys.csv', 'r') as datefile:
    reader = csv.DictReader(datefile)
    for row in reader:
        local_homes[row['fed_num']] = {'name': row['provider_name']}

In [83]:
with open('new_york_state_star_ratings_current.csv', 'r', encoding='iso-8859-1') as datefile:
    reader = csv.DictReader(datefile)
    for row in reader:
        if row['Federal Provider Number'] in local_homes:
            location_list = row['Location'].split('\n')
            local_homes[row['Federal Provider Number']]['location'] = location_list[0] + ' ' + location_list[1]
            local_homes[row['Federal Provider Number']]['ratings'] = {row['Processing Date']: {'overall_rate': row['Overall Rating'], 'qm_rate': row['QM Rating'], 'rn_staff_rate': row['RN Staffing Rating'], 'staff_rate': row['Staffing Rating'] }}

In [84]:
pprint(local_homes)

{'335056': {'location': '4459 BAILEY AVE AMHERST, NY 14226',
            'name': 'ELDERWOOD AT AMHERST',
            'ratings': {'6/1/18': {'overall_rate': '4',
                                   'qm_rate': '3',
                                   'rn_staff_rate': '4',
                                   'staff_rate': '4'}}},
 '335112': {'location': '10570 BERGTOLD ROAD CLARENCE, NY 14031',
            'name': 'BROTHERS OF MERCY NURSING & REHABILITATION CENTER',
            'ratings': {'6/1/18': {'overall_rate': '5',
                                   'qm_rate': '5',
                                   'rn_staff_rate': '4',
                                   'staff_rate': '4'}}},
 '335164': {'location': '64 HAGER STREET BUFFALO, NY 14208',
            'name': 'HUMBOLDT HOUSE REHABILITATION AND NURSING CENTER',
            'ratings': {'6/1/18': {'overall_rate': '1',
                                   'qm_rate': '4',
                                   'rn_staff_rate': '2',
                 

In [85]:
state_info = {}
with open('state_home_info.csv', 'r', encoding='iso-8859-1') as datefile:
    reader = csv.DictReader(datefile)
    for row in reader:
        medi_key = row['MEDICARE_NUMBER']
        if medi_key in local_homes:
            state_info[row['FACILITY_ID']] = row
            local_homes[medi_key]['name'] = row['FACILITY_NAME'].strip()
            local_homes[medi_key]['beds'] = int(row['TOTAL_BEDS'])

In [86]:
print(len(state_info) == len(local_homes))

True


In [87]:
medicaid_name_fixes = {'Odd Fellow & Rebekah Rehabilitation & Health Care Center Inc': 'Odd Fellow & Rebekah Rehabilitation&Health Care Center Inc', 'Williamsville Suburban LLC': 'Williamsville Suburban LLC', 'Brothers Of Mercy Nursing & Rehabilitation Center': 'Brothers of Mercy Nursing & Rehabilitation Center', 'Briody Rehab and Residential Health Care Center': 'Briody Rehab & Residential Health Care Center', 'Greenfield Health and Rehabilitation Center':'Greenfield Health & Rehab Center','Degraff Memorial Hospital-skilled Nursing Facility': 'Degraff Memorial Hospital-Skilled Nursing Facility', 'Beechwood  Homes': 'Beechwood Homes', 'Waterfront Center for Rehabilitation and Healthcare': 'Ellicott Center for Rehabilitation and Nursing'   }
#medicaid_name_fixes = {'Absolut Center for Nursing and Rehabilitation at Eden': 'ABSOLUT CTR FOR NURSING & REHAB EDEN L L C', 'Fiddlers Green Manor Rehabilitation and Nursing Center': 'FIDDLERS GREEN MANOR REHABILITATION & NURSING CTR', 'Absolut Center for Nursing and Rehabilitation at Auror': 'ABSOLUT CTR FOR NURSING & REHAB AURORA PARK L L C', 'Jennie B Richmond Chaffee Nursing Home Company Inc': 'JENNIE B RICHMOND CHAFFEE NURSING HOME CO., INC.', 'Comprehensive Rehabilitation and Nursing Center at Williamsville': 'COMPREHENSIVE REHAB & NURSING CTR AT WILLIAMSVILLE', 'Safire Rehabilitation of Northtowns LLC': 'SAFIRE REHABILITATION OF NORTHTOWNS, L L C', 'Safire Rehabilitation of Southtowns LLC': 'SAFIRE REHABILITATION OF SOUTHTOWN, L L C', 'Absolut Center for Nursing and Rehabilitation at Orcha': 'ABSOLUT CTR FOR NURSING & REHAB ORCHARD PARK L L C', 'Newfane Rehab & Health Care Center': 'NEWFANE REHAB AND HEALTH CARE CENTER', 'Absolut Center for Nursing and Rehabilitation at Gaspo': 'ABSOLUT CTR FOR NURSING & REHAB GASPORT L L C', 'Beechwood  Homes': 'BEECHWOOD HOMES'}

In [88]:
found_names = 0
found_keys = []
with open('medicaid_private_pay_2016.csv', 'r', encoding='iso-8859-1') as datefile:
    reader = csv.DictReader(datefile)
    for row in reader:
        if row['Name'] != '':
            if row['Name'] in medicaid_name_fixes:
                for home, info in local_homes.items():
                    if medicaid_name_fixes[row['Name']] == info['name']:
                        found_names += 1
                        found_keys.append(home)
                        local_homes[home]['private_days'] = float(row['% of Total Days Paid for by Private Pay '][:-1])
                        break
            else:
                if ' LLC' in row['Name']:
                    name_check = row['Name'].replace(' LLC', ', LLC')
                else:
                    name_check = row['Name']
                for home, info in local_homes.items():
                    if name_check in info['name']:
                        found_names += 1
                        found_keys.append(home)
                        local_homes[home]['private_days'] = float(row['% of Total Days Paid for by Private Pay '][:-1])
                        break
print(found_names)
for home, info in local_homes.items():
    if home not in found_keys:
        print(info['name'])

47


In [89]:
with open('state_fines.csv', 'r', encoding='iso-8859-1') as datefile:
    reader = csv.DictReader(datefile)
    for row in reader:
        if row['FACILITY_ID'] in state_info:
            medi_key = state_info[row['FACILITY_ID']]['MEDICARE_NUMBER']
            strp_date = datetime.strptime(row['STIP_DATE'], '%m/%d/%y 0:00')
            if strp_date > datetime(2012, 12,31):
                if 'state_fines' in local_homes[medi_key]:
                    local_homes[medi_key]['state_fines']['total'] += int(row['FINE_ASSESSED'])
                    local_homes[medi_key]['state_fines']['detail'].append({'amt': int(row['FINE_ASSESSED']), 'survey_date': row['SURVEY_DATE'], 'category': row['DEFICIENCY_CATEGORY']})
                else:
                    local_homes[medi_key]['state_fines'] = {'total': int(row['FINE_ASSESSED']),'detail':[{'amt': int(row['FINE_ASSESSED']), 'survey_date': row['SURVEY_DATE'], 'category': row['DEFICIENCY_CATEGORY']}]}

In [90]:
with open('medicare_fines.csv', 'r', encoding='iso-8859-1') as datefile:
    reader = csv.DictReader(datefile)
    for row in reader:
        medi_key = row['Federal Provider Number']
        if medi_key in local_homes:
            if row['Fine Amount'] != '':
                strp_date = datetime.strptime(row['Penalty Date'], '%m/%d/%y')
                if strp_date > datetime(2012, 12, 31):
                    clean_value = int(row['Fine Amount'][1:].replace(',',''))
                    if 'federal_fines' in local_homes[medi_key]:
                        local_homes[medi_key]['federal_fines']['total'] += clean_value
                        local_homes[medi_key]['federal_fines']['detail'].append({'amt': clean_value, 'survey_date': row['Penalty Date']})
                    else:
                        local_homes[medi_key]['federal_fines'] = {'total': clean_value,'detail':[{'amt': clean_value, 'survey_date': row['Penalty Date']}]}

In [91]:
with open('bedsores.csv', 'r', encoding='iso-8859-1') as datefile:
    reader = csv.DictReader(datefile)
    for row in reader:
        if row['Measure Code'] == '403':
            medi_key = row['Federal Provider Number']
            if medi_key in local_homes:
                local_homes[medi_key]['bedsores'] = float(row['Four Quarter Average Score'][:-1])

In [92]:
owners_na = 0
with open('medicaid_owners.csv', 'r', encoding='iso-8859-1') as datefile:
    reader = csv.DictReader(datefile)
    for row in reader:
        try:
            medi_key = row['Federal Provider Number']
        except KeyError:
            medi_key = row['PROVNUM']
        if medi_key in local_homes:
            try:
                category = row['Role played by Owner or Manager in Facility']
            except KeyError:
                category = row['ROLE_DESC']
            cat_key = titlecase(category)
            if category == 'Ownership Data Not Available':
                local_homes[medi_key]['owners'] = 'NA'
                owners_na += 1
                continue
            try:
                owner_name = row['Owner Name']
            except KeyError:
                owner_name = row['OWNER_NAME']
            try:
                if '%' in row['Ownership Percentage']:
                    owner_name += ' ({0})'.format(row['Ownership Percentage'])
            except KeyError:
                if '%' in row['OWNER_PERCENTAGE']:
                    owner_name += ' ({0})'.format(row['OWNER_PERCENTAGE'])
            if 'owners' in local_homes[medi_key]:
                if cat_key in local_homes[medi_key]['owners']:
                    local_homes[medi_key]['owners'][cat_key].append(owner_name)
                else:
                    local_homes[medi_key]['owners'][cat_key] = [owner_name]
            else:
                local_homes[medi_key]['owners'] = {cat_key: [owner_name]}
print('{0} ownership data missing'.format(owners_na))

0 ownership data missing


In [93]:
with open('owner_dates.csv', 'r', encoding='iso-8859-1') as datefile:
    reader = csv.DictReader(datefile)
    for row in reader:
        local_homes[row['fed_key']]['name'] = row['fed_name']
        local_homes[row['fed_key']]['sale_date'] = row['owner_date']
        local_homes[row['fed_key']]['cond_local'] = row['cond_local'] == 'Y'
        local_homes[row['fed_key']]['notes'] = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut quis velit velit. Ut eget placerat dolor. Fusce nisi lacus, malesuada in pulvinar a, sagittis nec ex. Maecenas facilisis pharetra diam, non sollicitudin massa euismod eleifend. Sed condimentum fermentum ligula nec maximus. Aenean eu elit tempor, elementum ligula sed, placerat nunc. Fusce luctus gravida sapien ut venenatis. Cras quis sollicitudin turpis, sit amet convallis orci.'#row['note']

In [94]:
years = list(range(2009, 2014))
for year in years:
    found_homes = 0
    file_name = 'star_ratings_{0}.csv'.format(year)
    with open(file_name, 'r', encoding='iso-8859-1') as datefile:
        reader = csv.DictReader(datefile)
        for row in reader:
            med_key = row['PROVNUM']
            if med_key in local_homes:
                processed_date = '{0}/1/{1}'.format(row['month'],row['year'][2:])
                star_rating = row['overall_rating'][0:1]
                local_homes[med_key]['ratings'][processed_date] = {'overall_rate': star_rating}
                found_homes += 1
    print('{0} found homes in {1}'.format((found_homes/12), year))

47.0 found homes in 2009
47.0 found homes in 2010
47.0 found homes in 2011
47.0 found homes in 2012
47.0 found homes in 2013


In [95]:
months = list(range(1,13))
print(months)
years = list(range(2014, 2019))
print(years)
for year in years:
    for month in months:
        found_homes = 0
        if month == 12 and year == 2014:
            file_name = 'monthly/Nursing home ratings {0}-{1}.csv'.format(month,year)
        elif month == 6 and year == 2018:
            print('That\'s all folks')
            break
        else:
            file_name = 'monthly/Nursing home ratings {0}-{1}.xlsx.csv'.format(month,year)  
        with open(file_name, 'r', encoding='iso-8859-1') as datefile:
            reader = csv.DictReader(datefile)
            for row in reader:
                try:
                    med_key = row['provnum']
                except KeyError:
                    med_key = row['PROVNUM']
                if med_key in local_homes:
                    processed_date = "{0}/1/{1}".format(month, str(year)[2:])
                    try:
                        star_rating = row['overall_rating']
                    except KeyError:
                        star_rating = row['OVERALL_RATING']
                    local_homes[med_key]['ratings'][processed_date] = {'overall_rate': star_rating}
                    found_homes += 1
        if found_homes != len(local_homes):
            print('{0} found homes in {2}-{1}'.format(found_homes, year, month))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[2014, 2015, 2016, 2017, 2018]
That's all folks


In [96]:
pprint(local_homes['335663'])

{'beds': 120,
 'bedsores': 9.65,
 'cond_local': False,
 'federal_fines': {'detail': [{'amt': 85925, 'survey_date': '5/19/16'}],
                   'total': 85925},
 'location': '300 DORRANCE AVENUE BUFFALO, NY 14220',
 'name': 'Safire Rehabilitation of Southtowns, LLC',
 'notes': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut quis '
          'velit velit. Ut eget placerat dolor. Fusce nisi lacus, malesuada in '
          'pulvinar a, sagittis nec ex. Maecenas facilisis pharetra diam, non '
          'sollicitudin massa euismod eleifend. Sed condimentum fermentum '
          'ligula nec maximus. Aenean eu elit tempor, elementum ligula sed, '
          'placerat nunc. Fusce luctus gravida sapien ut venenatis. Cras quis '
          'sollicitudin turpis, sit amet convallis orci.',
 'owners': {'5% or Greater Indirect Ownership Interest': ['SAFIRE '
                                                          'REHABILITATION OF '
                                                  

In [97]:
homes_chart = []
#options = ['1', '2', '3', '4', '5']
for home, info in local_homes.items():
    item = {'fpn': home, 'name': info['name'], 'values': [], 'sale_date': info['sale_date']}
    dates = sorted(info['ratings'].keys())
    """item['values'].append({'date': '1/1/13', 'overall_rate': random.choice(options)})
    item['values'].append({'date': '1/1/14', 'overall_rate': random.choice(options)})
    item['values'].append({'date': '1/1/15', 'overall_rate': random.choice(options)})
    item['values'].append({'date': '1/1/16', 'overall_rate': random.choice(options)})
    item['values'].append({'date': '1/1/17', 'overall_rate': random.choice(options)})"""
    for dated in dates:
        new_dict = info['ratings'][dated]
        new_dict['date'] = dated
        item['values'].append(new_dict)
    homes_chart.append(item)

In [98]:
sorted_chart = sorted(homes_chart,key=lambda x: x['name'])

In [99]:
chart_export = {'houses': sorted_chart}
with open('chart_export.json', 'w') as datafile:
    json.dump(chart_export, datafile)

In [100]:
for home, info in local_homes.items():
    new_ratings = {'6/1/18':{}}
    new_ratings['6/1/18'] = info['ratings']['6/1/18']
    local_homes[home]['ratings'] = new_ratings
pprint(local_homes['335663'])

{'beds': 120,
 'bedsores': 9.65,
 'cond_local': False,
 'federal_fines': {'detail': [{'amt': 85925, 'survey_date': '5/19/16'}],
                   'total': 85925},
 'location': '300 DORRANCE AVENUE BUFFALO, NY 14220',
 'name': 'Safire Rehabilitation of Southtowns, LLC',
 'notes': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut quis '
          'velit velit. Ut eget placerat dolor. Fusce nisi lacus, malesuada in '
          'pulvinar a, sagittis nec ex. Maecenas facilisis pharetra diam, non '
          'sollicitudin massa euismod eleifend. Sed condimentum fermentum '
          'ligula nec maximus. Aenean eu elit tempor, elementum ligula sed, '
          'placerat nunc. Fusce luctus gravida sapien ut venenatis. Cras quis '
          'sollicitudin turpis, sit amet convallis orci.',
 'owners': {'5% or Greater Indirect Ownership Interest': ['SAFIRE '
                                                          'REHABILITATION OF '
                                                  

In [101]:
pprint(local_homes['335577'])

{'beds': 96,
 'bedsores': 0.46,
 'cond_local': True,
 'location': '1818 COMO PARK BLVD LANCASTER, NY 14086',
 'name': 'Elderwood at Lancaster',
 'notes': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut quis '
          'velit velit. Ut eget placerat dolor. Fusce nisi lacus, malesuada in '
          'pulvinar a, sagittis nec ex. Maecenas facilisis pharetra diam, non '
          'sollicitudin massa euismod eleifend. Sed condimentum fermentum '
          'ligula nec maximus. Aenean eu elit tempor, elementum ligula sed, '
          'placerat nunc. Fusce luctus gravida sapien ut venenatis. Cras quis '
          'sollicitudin turpis, sit amet convallis orci.',
 'owners': {'5% or Greater Direct Ownership Interest': ['COLE, WARREN (50%)',
                                                        'RUBIN, JEFFREY (50%)'],
            'Managing Employee': ['BOTHWELL, DENISE'],
            'Operational/Managerial Control': ['COLE, WARREN',
                                               

In [102]:
with open('data.json', 'w') as datafile:
    json.dump(local_homes, datafile)